In [1]:
import numpy as np

In [2]:
def softmax(Z):
    Z = Z - np.max(Z, axis = 0, keepdims = True)
    e = np.exp(Z)
    d = np.sum(e, axis = 0, keepdims = True)
    return e / d

In [3]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

In [4]:
def cross_entropy_loss(y, a):
    return -np.sum(y * np.log(a))

In [5]:
def cross_entropy_loss_many(Y, A):
    m = Y.shape[1]
    J = -np.sum(Y * np.log(np.maximum(A, 1e-10)), axis = 0, keepdims = True)
    return np.sum(J) / m

In [6]:
def numeric_jacobian(x, f, epsilon=1e-4):
    x = np.copy(x)
    a = f(x)
    a_shape = () if np.isscalar(a) else a.shape
    result = np.zeros(a_shape + x.shape)
    for index, y in np.ndenumerate(x):
        y_prev = y - epsilon
        y_next = y + epsilon
        x[index] = y_prev
        f_prev = f(x)
        x[index] = y_next
        f_next = f(x)
        x[index] = y
        f_grad = (f_next - f_prev) / (2 * epsilon)
        result[(...,) + index] = f_grad
    return result

In [7]:
def numeric_jacobian_test(x, f, expected, epsilon=1e-4):
    j = numeric_jacobian(x, f, epsilon)
    print('analytic:')
    print(expected)
    print()
    print('numeric:')
    print(j)
    print()
    print('error norm:', np.linalg.norm(expected - j))

In [8]:
z = np.array([1.,2.,3.])
f = sigmoid
a = f(z)
dz = np.diag(a * (1 - a))

numeric_jacobian_test(z, f, dz)

analytic:
[[ 0.19661193  0.          0.        ]
 [ 0.          0.10499359  0.        ]
 [ 0.          0.          0.04517666]]

numeric:
[[ 0.19661193  0.          0.        ]
 [ 0.          0.10499359  0.        ]
 [ 0.          0.          0.04517666]]

error norm: 1.03863334212e-10


In [9]:
z = np.array([1.,2.,3.])
f = np.tanh
a = f(z)
dz = np.diag(1 - np.power(a, 2))

numeric_jacobian_test(z, f, dz)

analytic:
[[ 0.41997434  0.          0.        ]
 [ 0.          0.07065082  0.        ]
 [ 0.          0.          0.00986604]]

numeric:
[[ 0.41997434  0.          0.        ]
 [ 0.          0.07065083  0.        ]
 [ 0.          0.          0.00986604]]

error norm: 1.12035346308e-09


In [10]:
z = np.array([1.,2.,3.])
f = softmax
a = f(z)
dz = np.diag(a) - np.outer(a, a)

numeric_jacobian_test(z, f, dz)

analytic:
[[ 0.08192507 -0.02203304 -0.05989202]
 [-0.02203304  0.18483645 -0.1628034 ]
 [-0.05989202 -0.1628034   0.22269543]]

numeric:
[[ 0.08192507 -0.02203304 -0.05989202]
 [-0.02203304  0.18483645 -0.1628034 ]
 [-0.05989202 -0.1628034   0.22269543]]

error norm: 1.86126090163e-10


In [11]:
y = np.array([0.2, 0.0, 0.8])
a = np.array([0.1, 0.3, 0.6])
f = lambda a: cross_entropy_loss(y, a)
da = - y / a

numeric_jacobian_test(a, f, da)

analytic:
[-2.         -0.         -1.33333333]

numeric:
[-2.00000067  0.         -1.33333335]

error norm: 6.66781910961e-07


In [12]:
y = np.array([0.2, 0.0, 0.8])
z = np.array([1., 2., 3.])
a = softmax(z)
f = lambda z: cross_entropy_loss(y, softmax(z))
dz = a - y

numeric_jacobian_test(z, f, dz)

analytic:
[-0.10996943  0.24472847 -0.13475904]

numeric:
[-0.10996943  0.24472847 -0.13475904]

error norm: 2.29340228256e-10


In [22]:
W = np.random.rand(3, 4)
b = np.random.rand(3)
x = np.random.rand(4)
z = np.matmul(W, x) + b
a = softmax(z)
y = np.array([0.2, 0.0, 0.8])

print('dJ/dW')
print('=====')
print()

f = lambda W: cross_entropy_loss(y, softmax(np.matmul(W, x) + b))
dz = a - y
dW = np.outer(a - y, x)
numeric_jacobian_test(W, f, dW)

print()
print('dJ/dx')
print('=====')
print()

f = lambda x: cross_entropy_loss(y, softmax(np.matmul(W, x) + b))
dz = a - y
dx = np.matmul(dz.reshape(1, dz.size), W)
numeric_jacobian_test(x, f, dx)

print()
print('dJ/db')
print('=====')
print()

f = lambda b: cross_entropy_loss(y, softmax(np.matmul(W, x) + b))
dz = a - y
db = dz
numeric_jacobian_test(b, f, db)

dJ/dW
=====

analytic:
[[ 0.05835788  0.23767355  0.15065267  0.07192808]
 [ 0.06459683  0.26308287  0.16675872  0.07961781]
 [-0.12295471 -0.50075643 -0.31741138 -0.15154589]]

numeric:
[[ 0.05835788  0.23767355  0.15065267  0.07192808]
 [ 0.06459683  0.26308287  0.16675872  0.07961781]
 [-0.12295471 -0.50075643 -0.31741138 -0.15154589]]

error norm: 1.89989372869e-10

dJ/dx
=====

analytic:
[[ 0.15249132  0.37423959 -0.09565523 -0.20249029]]

numeric:
[ 0.15249132  0.37423959 -0.09565523 -0.20249029]

error norm: 4.76952760471e-11

dJ/db
=====

analytic:
[ 0.25024567  0.27699905 -0.52724472]

numeric:
[ 0.25024567  0.27699905 -0.52724472]

error norm: 2.15040108902e-10


In [19]:
W = np.random.rand(3, 4)
b = np.random.rand(3, 1)
X = np.random.rand(4, 2)
Z = np.matmul(W, X) + b
A = softmax(Z)
Y = softmax(np.random.rand(3, 2))

print('dJ/dW')
print('=====')
print()

f = lambda W: cross_entropy_loss_many(Y, softmax(np.matmul(W, X) + b))
dZ = (A - Y) / Y.shape[1]
dW = np.matmul(dZ, X.T)
numeric_jacobian_test(W, f, dW)

print()
print('dJ/dX')
print('=====')
print()

f = lambda X: cross_entropy_loss_many(Y, softmax(np.matmul(W, X) + b))
dZ = (A - Y) / Y.shape[1]
dX = np.matmul(W.T, dZ)
numeric_jacobian_test(X, f, dX)

print()
print('dJ/db')
print('=====')
print()

f = lambda b: cross_entropy_loss_many(Y, softmax(np.matmul(W, X) + b))
dZ = (A - Y) / Y.shape[1]
db = np.matmul(dZ, np.ones((2, 1)))
numeric_jacobian_test(b, f, db)

dJ/dW
=====

analytic:
[[-0.04022874 -0.0159691  -0.07395404 -0.06597514]
 [-0.10300245 -0.05765084 -0.15381043 -0.11379874]
 [ 0.1432312   0.07361994  0.22776446  0.17977388]]

numeric:
[[-0.04022874 -0.0159691  -0.07395404 -0.06597514]
 [-0.10300245 -0.05765084 -0.15381043 -0.11379874]
 [ 0.1432312   0.07361994  0.22776446  0.17977388]]

error norm: 2.08093198502e-10

dJ/dX
=====

analytic:
[[-0.00299724  0.01108631]
 [-0.00973512 -0.06479612]
 [ 0.03534242  0.15264563]
 [ 0.00425963  0.05495008]]

numeric:
[[-0.00299724  0.01108631]
 [-0.00973512 -0.06479612]
 [ 0.03534242  0.15264563]
 [ 0.00425963  0.05495008]]

error norm: 3.69815593006e-11

dJ/db
=====

analytic:
[[-0.08386111]
 [-0.17826072]
 [ 0.26212183]]

numeric:
[[-0.08386111]
 [-0.17826072]
 [ 0.26212183]]

error norm: 2.23671954153e-10
